In [1]:
import json
import re

Построковое чтение JSON-файла:

In [2]:
with open('hw_3_twitter.json', 'r') as f:
    lines = f.read().split('\n')

In [3]:
data = []
for line in lines:
    if line != '':
        data.append(json.loads(line))

Сколько твитов в наборе?

In [4]:
len(data)

2556

In [5]:
not_deleted = [d for d in data if 'delete' not in d]

Какой процент твитов составляют удалённые записи (помеченные как deleted)?

In [6]:
100 - len(not_deleted) / len(data) * 100

14.162754303599385

Какие самые популярные языки твитов?

In [7]:
langs = {}
for d in not_deleted:
    langs[d['lang']] = langs.get(d['lang'], 0) + 1
top_langs = sorted(langs, key=lambda x: -langs[x])[:3]
print({lang: langs[lang] for lang in top_langs[:3]})

{'en': 719, 'ja': 438, 'es': 173}


Есть ли твиты от одного и того же пользователя? Если да, то сколько таких пользователей?

In [8]:
user_tweets = {}
for d in not_deleted:
    user_tweets[d['user']['id']] = user_tweets.get(d['user']['id'], 0) + 1
print(sum(v > 1 for v in user_tweets.values()))

25


Топ-20 хэштегов:

In [9]:
hashtags = {}
for d in not_deleted:
    for ht in d['entities']['hashtags']:
        hashtags[ht['text']] = hashtags.get(ht['text'], 0) + 1
top_hashtags = sorted(hashtags, key=lambda x: -hashtags[x])
print(*{ht: hashtags[ht] for ht in top_hashtags[:20]}.items(), sep='\n')

('BTS', 17)
('방탄소년단', 13)
('AMAs', 11)
('人気投票ガチャ', 8)
('태형', 7)
('뷔', 6)
('BTSinChicago', 5)
('BTSLoveYourselfTour', 5)
('오늘의방탄', 5)
('PledgeForSwachhBharat', 5)
('MPN', 5)
('PCAs', 4)
('V', 4)
('시카고1회차공연', 4)
('เป๊กผลิตโชค', 4)
('JIMIN', 4)
('running', 3)
('NCT', 3)
('지민', 3)
('WajahmuPlastik', 3)


Предобработать тексты оригинальных твитов (не ретвитов) на английском языке, убрать пунктуацию, привести к нижнему регистру и составить частотный словарь

In [10]:
texts = []
freq_dict = {}
for d in not_deleted:
    if d['lang'] == 'en':
        if d['retweeted'] != 'false':
            texts.append(re.split('[^\'\w]+', d['text'].lower()))
            for word in texts[-1]:
                freq_dict[word] = freq_dict.get(word, 0) + 1
print(*texts, sep='\n')

['rt', 'elissesifieds', 'nothing', 'can', 'stop', 'us', 'from', 'supporting', 'you', 'when', 'we', 'say', 'all', 'the', 'way', 'it', 'will', 'be', 'indeed', 'hello', 'elissesifieds', 'cebu', 'thank', 'yo', '']
['rt', 'riverdalencisla', 'i', 'vote', 'for', 'shadowhunters', 'for', 'theshow', 'of', '2018', 'in', 'the', 'e', 'people', 's', 'choice', 'awards', 'pcas', 'saveshadowhunters', '']
['rt', 'corruptedray001', 'don', 't', 'try', 'coming', 'around', 'when', 'i', 'no', 'longer', 'want', 'u', 'around']
['', 'tatolote', 'higher', 'than', '600']
['rt', 'ahmedtwinkiee', 'send', 'her', 'back', 'pics', 'https', 't', 'co', 'pkye51rzna']
['time', 'to', 'think', 'about', 'what', 'toys', "you'll", 'be', 'getting', 'for', 'your', 'kids', 'this', 'christmas', 'here', 'are', 'our', 'top', 'picks', 'https', 't', 'co', 'bg7jq6ap6d']
['rt', 'naveedajamali', 'a', 'background', 'investigation', 'means', 'that', 'all', 'the', 'things', 'said', 'by', 'those', 'interviewed', 'will', 'exist', 'without', 'a

In [11]:
print(*('{} => {}'.format(k, v) for k, v in freq_dict.items()), sep='\n')

rt => 395
elissesifieds => 2
nothing => 5
can => 34
stop => 5
us => 8
from => 28
supporting => 1
you => 149
when => 31
we => 36
say => 7
all => 37
the => 267
way => 12
it => 98
will => 19
be => 56
indeed => 1
hello => 1
cebu => 1
thank => 8
yo => 1
 => 342
riverdalencisla => 1
i => 187
vote => 8
for => 98
shadowhunters => 1
theshow => 1
of => 117
2018 => 11
in => 107
e => 4
people => 19
s => 60
choice => 2
awards => 1
pcas => 3
saveshadowhunters => 1
corruptedray001 => 1
don => 14
t => 430
try => 3
coming => 6
around => 5
no => 21
longer => 2
want => 15
u => 13
tatolote => 1
higher => 3
than => 9
600 => 1
ahmedtwinkiee => 1
send => 1
her => 17
back => 12
pics => 1
https => 382
co => 380
pkye51rzna => 1
time => 22
to => 187
think => 10
about => 28
what => 40
toys => 2
you'll => 3
getting => 6
your => 30
kids => 4
this => 93
christmas => 1
here => 8
are => 48
our => 21
top => 3
picks => 1
bg7jq6ap6d => 1
naveedajamali => 1
a => 163
background => 2
investigation => 3
means => 2
that => 76

Найти количество подписчиков (фолловеров) у авторов твитов и вывести топ-10

In [12]:
users = {}
for d in not_deleted:
    users[d['user']['screen_name']] = d['user']['followers_count']
top_users = sorted(users, key=lambda x: -users[x])
print(*{u: users[u] for u in top_users[:10]}.items(), sep='\n')

('FIL0S0FIA', 2521403)
('FitnessMagazine', 1491309)
('malaysiakini', 1206759)
('NYTScience', 1137374)
('GramaticaReal', 625463)
('tgrthabertv', 392472)
('TheSunFootball', 383698)
('Melbourne', 374222)
('Roznama_Express', 318189)
('burger_boogie', 311319)
